In [1]:
from transformers import pipeline
import spacy
import en_core_web_sm
import nltk
from goose3 import Goose
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from rouge import Rouge

In [2]:
summarizor = pipeline('summarization', model = 'facebook/bart-large-cnn')

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

In [4]:
tokenizer.model_max_length

1024

In [5]:
file = "C:\\Lachin\\NLP Project\\Full podcast.txt"
with open(file,'r') as f:
    original_text = f.readlines()
original_text = ''.join(original_text)

In [6]:
file1 = "C:\\Lachin\\NLP Project\\reference_summary.txt"
with open(file1,'r') as f1:
    reference_summary = f1.readlines()
reference_summary = ''.join(reference_summary)

In [7]:
def summarizing_text(text, number_of_tokens = tokenizer.model_max_length, max_len=0, min_len=0, human_summary=reference_summary):
    sentences = []
    total_sentences = []
    length = 0
    original_sentences = nltk.tokenize.sent_tokenize(text)
    for sent in original_sentences:
        word_tokens = tokenizer(sent)
        length += len(word_tokens['input_ids'])
        if (length < number_of_tokens):
            sentences.append(sent)
        else:
            total_sentences.append(sentences)
            length = len(word_tokens['input_ids'])
            sentences = []
            sentences.append(sent)
    
    
    #Summarizing the text now
    system_summary = []
    for i in range(len(total_sentences)):
        if (i==0):
            print(len(total_sentences))
        print(i)
        text1 = ' '.join(total_sentences[i])
        if ((max_len==0) and (min_len==0)):
            z = summarizor(text1, do_sample=False)
            system_summary.append(z[0]['summary_text'])
        elif((max_len >0) and (min_len > 0)):
            z = summarizor(text1, max_length=max_len, min_length =min_len, do_sample=False)
            system_summary.append(z[0]['summary_text'])
        else:
            print('Please check "max_len" and "min_len" parameters')
    final_system_summary = " ".join(system_summary)
    final_summary_tokens = tokenizer(final_system_summary)
    print('Number of tokens in system summary {}'.format(len(final_summary_tokens['input_ids'])))
    human_summary_tokens = tokenizer(human_summary)
    print('Number of tokens in human summary {}'.format(len(human_summary_tokens['input_ids'])))
    
    #Accuracy checking
    r = Rouge()
    results = r.get_scores(final_system_summary,human_summary)
    print(results)
    
    
    print(final_system_summary)

In [8]:
%%time
summarizing_text(original_text,number_of_tokens=1024, max_len=500, min_len=300)

6
0
1
2
3
4
5


Token indices sequence length is longer than the specified maximum sequence length for this model (1879 > 1024). Running this sequence through the model will result in indexing errors


Number of tokens in system summary 1879
Number of tokens in human summary 521
[{'rouge-1': {'r': 0.6832579185520362, 'p': 0.2926356589147287, 'f': 0.40976933094355533}, 'rouge-2': {'r': 0.412987012987013, 'p': 0.1260904044409199, 'f': 0.19319562217559713}, 'rouge-l': {'r': 0.6380090497737556, 'p': 0.27325581395348836, 'f': 0.38263228888114015}}]
Harj Taggar: Why should you have a co-founder when you're starting a company? He says there are three main reasons: productivity, moral support and pattern matching. He says Apple, Facebook, Google, Microsoft all had co-founders when they started. Harj: If you're keen to get started in a company now, and you don't have a great co- founder that immediately available, or someone that you can bring on immediately, go ahead and get started and get a company started. But if you can't wait, or you're not sure if you want to wait, then you should get started now and get your company started with a good co-Founder, says Harj, who is also a partner at Y

In [9]:
%%time
summarizing_text(original_text)

6
0
1
2
3
4
5
Number of tokens in system summary 429
Number of tokens in human summary 521
[{'rouge-1': {'r': 0.416289592760181, 'p': 0.47668393782383417, 'f': 0.4444444394673155}, 'rouge-2': {'r': 0.15844155844155844, 'p': 0.1942675159235669, 'f': 0.17453504512311696}, 'rouge-l': {'r': 0.38009049773755654, 'p': 0.43523316062176165, 'f': 0.40579709647214646}}]
Harj Taggar: Why should you have a co-founder when you're starting a company? He says there are three main reasons: productivity, moral support and pattern matching. He says Apple, Facebook, Google, Microsoft all had co-founders when they started. Harj: If you're keen to get started in a company now, and you don't have a great co- founder, go ahead and get started. Y Combinator has funded single founders, and we have done so successfully. But there are some exceptions to this rule and they tend to follow a particular pattern. The single founders have such conviction in the idea that they don't need a co-founder to convince them t

In [10]:
%%time
summarizing_text(original_text,number_of_tokens=1024, max_len=200, min_len=100)

6
0
1
2
3
4
5
Number of tokens in system summary 644
Number of tokens in human summary 521
[{'rouge-1': {'r': 0.49321266968325794, 'p': 0.42412451361867703, 'f': 0.45606694063505543}, 'rouge-2': {'r': 0.2077922077922078, 'p': 0.17167381974248927, 'f': 0.18801409610287767}, 'rouge-l': {'r': 0.4343891402714932, 'p': 0.3735408560311284, 'f': 0.4016736351957249}}]
Harj Taggar: Why should you have a co-founder when you're starting a company? He says there are three main reasons: productivity, moral support and pattern matching. He says Apple, Facebook, Google, Microsoft all had co-founders when they started. Harj: If you're keen to get started in a company now, and you don't have a great co- founder that immediately available, or someone that you can bring on, go ahead and get started. Y Combinator has funded single founders, and we have done so successfully. But there are some exceptions to this rule and they tend to follow a particular pattern. If you're really keen to get a startup right

In [11]:
%%time
summarizing_text(original_text, number_of_tokens=500)

14
0
1
2
3
4
5
6
7
8
9
10
11
12
13
Number of tokens in system summary 1005
Number of tokens in human summary 521
[{'rouge-1': {'r': 0.5520361990950227, 'p': 0.33516483516483514, 'f': 0.4170940123927826}, 'rouge-2': {'r': 0.24935064935064935, 'p': 0.13351877607788595, 'f': 0.17391303893590185}, 'rouge-l': {'r': 0.497737556561086, 'p': 0.3021978021978022, 'f': 0.3760683713671415}}]
Harj Taggar: Why should you have a co-founder at all? He says there are three main reasons: productivity, moral support and brainstorming. He says the best co-founding relationships help balance out the emotional rollercoaster that is a startup. Taggar is a partner at Y Combinator. You don't get that same emotional support that you do from a co-founder. Anyone who's working for you, you're the boss, and they're not as invested in the success of the startup as you are. And you can't really be as open or honest with them about the company as you can be with a peer. The final reason I'd say to have a Co-founder i

In [ ]:
%%time
summarizing_text(original_text, number_of_tokens=500, max_len=80, min_len=40)